#### Import libraries

In [ ]:
import os
import time
import gym
import numpy as np
from tqdm.notebook import trange, tqdm
from IPython.display import clear_output

#### Define the environment, initialize Q-Table and hyperparameters

In [ ]:
# Define the environment
env = gym.make('Taxi-v3')

# Initialize the q-table with zero values
q_table = np.zeros([env.observation_space.n, env.action_space.n])

# Hyperparameters
alpha = 0.1  # learning-rate
gamma = 0.7  # discount-factor
epsilon = 0.1  # explor vs exploit

#### Q-Learning phase

In [ ]:
# For plotting metrics
all_epochs = []
all_penalties = []

seed= 42
rng =np.random.default_rng(seed)

for i in range(1, 100001):
    state = env.reset()

    epochs, penalties, reward, = 0, 0, 0
    done = False

    while not done:
        if rng.random() < epsilon:
            action = env.action_space.sample() # Explore action space
        else:
            action = np.argmax(q_table[state]) # Exploit learned values

        next_state, reward, done, info = env.step(action)

        old_value = q_table[state, action]
        next_max = np.max(q_table[next_state])

        new_value = (1 - alpha) * old_value + alpha * (reward + gamma * next_max)
        q_table[state, action] = new_value

        if reward == -10:
            penalties += 1

        state = next_state
        epochs += 1

    if i % 1000 == 0:
        clear_output(wait=True)
        print(f"Episode: {i}")

print("Training finished.\n")

#### Evaluate agent performance after Q-Learning

In [ ]:
total_epochs, total_penalties = 0, 0
episodes = 100

for _ in range(episodes):
    state = env.reset()  # reset environment to a new, random state
    epochs, penalties, reward = 0, 0, 0

    done = False

    while not done:
        action = np.argmax(q_table[state])
        state, reward, done, info = env.step(action)

        if reward == -10:
            penalties += 1

        epochs += 1

    total_penalties += penalties
    total_epochs += epochs

print(f"Results after {episodes} episodes:")
print(f"Average timesteps per episode: {total_epochs / episodes}")
print(f"Average penalties per episode: {total_penalties / episodes}")

#### Play an episode

In [ ]:
state = env.reset()  # reset environment to a new, random state
env.render()
time.sleep(0.5)
done = False

while not done:
    action = np.argmax(q_table[state])
    state, reward, done, info = env.step(action)
    clear_output(wait=True)
    env.render()
    print(reward)
    time.sleep(0.5)

In [ ]:
q_table